In [1]:
from typing import List, Optional, Generator
import typing
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Atcoder Beginner contest: 365

In [4]:
def leap_year(year: int) -> int:
    if year % 4 != 0:
        return 365
    elif year % 100 != 0:
        return 366
    elif year % 400 != 0:
        return 365
    else:
        return 366

# Testing the solution
assert leap_year(2023) == 365
assert leap_year(1992) == 366
assert leap_year(1800) == 365
assert leap_year(1600) == 366

# %$
def second_largest(N: int, nums: list[int]) -> int:
    max_ = max(nums)
    idx = 0 if nums[0] != max_ else 1
    for i in range(N):
        if nums[idx] < nums[i] < max_:
            idx = i
    return idx + 1

# Testing the solution
assert second_largest(5, [3,8,2,5,1]) == 4
assert second_largest(4, [8,2,5,1]) == 3
assert second_largest(2, [2,1]) == 2
assert second_largest(8, [1,2,3,4,5,10,9,11]) == 6

In [5]:
def transportation_expenses(N: int, M: int, costs: list[int]) -> int:
    total_cost = sum(costs)
    if total_cost <= M:
        return -1
    else:
        low, high = 0, M
        while low <= high:
            mid = (low + high) // 2
            total_cost = sum(map(lambda x: min(x, mid), costs))
            if total_cost <= M:
                low = mid + 1
            else:
                high = mid - 1

        return high

# Testing the solution
assert transportation_expenses(4, 8, [1,3,2,4]) == 2
assert transportation_expenses(3, 20, [5,3,2]) == -1
assert transportation_expenses(10, 23, [2,5,6,5,2,1,7,9,7,2]) == 2

In [6]:
def RPS(N: int, moves: str) -> int:

    win_mapping: dict[str, str] = {"R": "P", "S": "R", "P": "S"}
    move_mapping: dict[int, str] = dict(enumerate("RPS"))
    move_mapping_rev: dict[str, int] = {j: i for i, j in move_mapping.items()}

    dp: list[int] = [0 for j in range(3)]
    for i in range(N - 1, -1, -1):
        curr, win_move = moves[i], win_mapping[moves[i]]
        next_dp: list[int] = [0 for i in range(3)]
        for j in range(3):
            prev = move_mapping[j]
            if win_move != prev:
                next_dp[j] = max(next_dp[j], 1 + dp[move_mapping_rev[win_move]])
            if moves[i] != prev:
                next_dp[j] = max(next_dp[j], dp[move_mapping_rev[curr]])
        dp = next_dp

    return max(dp)

# Testing the solution
assert RPS(24, "SPRPSRRRRRPPRPRPSSRSPRSS") == 18
assert RPS(10, "SSSSSSSSSS") == 5
assert RPS(5, "SSPRS") == 4
assert RPS(6, "PRSSRS") == 5

#### Leetcode Biweekly

In [7]:
def minFlips(grid: list[list[int]]) -> int:
    M, N = len(grid), len(grid[0])

    # Count flips required horizontally
    h_flips = 0
    for i in range(M):
        j, k, flips = 0, N - 1, 0
        while j < k:
            if grid[i][j] != grid[i][k]:
                flips += 1
            j, k = j + 1, k - 1
        h_flips += flips

    # Count flips required vertically
    v_flips = 0
    for i in range(N):
        j, k, flips = 0, M - 1, 0
        while j < k:
            if grid[j][i] != grid[k][i]:
                flips += 1
            j, k = j + 1, k - 1
        v_flips += flips

    return min(h_flips, v_flips)

# Testing the solution
assert minFlips([[1,0,0],[0,0,0],[0,0,1]]) == 2
assert minFlips([[0,1],[0,1],[0,0]]) == 1
assert minFlips([[1],[0]]) == 0
assert minFlips([[1]]) == 0

#### Upsolving Min Flips 2

In [8]:
def minFlipsII(grid: list[list[int]]) -> int:
    # https://leetcode.com/problems/minimum-number-of-flips-to-make-binary-grid-palindromic-ii/solutions/5581114/counting-greedy-python3/
    def symmetric_pairs(i: int, j: int) -> set[tuple[int, int]]:
        return {(i, j), (i, N - j - 1), (M - i - 1, j), (M - i - 1, N - j - 1)}

    # Compute dimensions
    M, N = len(grid), len(grid[0])

    # Calculate the number of flips
    # 4 way symmetry - all 0s or all 1s
    # 2 way symmetry - based on odd / even count of (1, 1), (0, 1) / (1, 0) we decide
    # 1 cell symmetry - all 0s

    flips, tws10, tws11 = 0, 0, 0
    visited: set[tuple[int, int]] = set()
    for i in range(M):
        for j in range(N):
            if (i, j) not in visited:
                ones, pairs = 0, symmetric_pairs(i, j)
                for x, y in pairs:
                    if grid[x][y] == 1:
                        ones += 1
                    visited.add((x, y))

                # Four cell symmetry
                if len(pairs) == 4:
                    flips += min(ones, len(pairs) - ones)

                # Single cell symmetry
                elif len(pairs) == 1:
                    flips += ones

                # Two cell symmetry
                else:
                    # Count 1, 1
                    if ones == 2:
                        tws11 += 1
                    # Count 1, 0 / 0, 1
                    elif ones == 1:
                        tws10 += 1

    # If even number of 1, 1 pairs
    if tws11 % 2 == 0:
        # Change all (0, 1), (1, 0) to (0, 0)
        flips += tws10
    else:
        # If there is atleast a single (1, 0) pair change all (1, 0) that to (1, 1)
        # Otherwise change a single (1, 1) to (0, 0)
        flips += tws10 if tws10 else 2

    return flips

# Testing the solution
assert minFlipsII([[1,0,0],[0,1,0],[0,0,1]]) == 3
assert minFlipsII([[0,1],[0,1],[0,0]]) == 2
assert minFlipsII([[1],[1]]) == 2
assert minFlipsII([[1],[1],[1],[0]]) == 1
assert minFlipsII([[1,1,0,0,1],[0,1,1,0,1],[0,0,1,1,1],[1,0,1,1,0],[1,0,1,1,0],[0,0,1,1,1],[0,1,1,0,1],[1,1,0,1,0]]) == 16

LC Weekly - 4th Aug 2024

In [9]:
class neighborSum:

    def __init__(self, grid: List[List[int]]):
        self.grid: list[list[int]] = grid
        self.M, self.N = len(grid), len(grid[0])
        self.positions: dict[int, tuple[int, int]] = {grid[i][j]: (i, j) for i in range(self.M) for j in range(self.N)}

    def adjacentSum(self, value: int) -> int:
        result = 0
        i, j = self.positions[value]
        for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
            if 0 <= x < self.M and 0 <= y < self.N:
                result += self.grid[x][y]
        return result

    def diagonalSum(self, value: int) -> int:
        result = 0
        i, j = self.positions[value]
        for x, y in ((i - 1, j - 1), (i + 1, j + 1), (i - 1, j + 1), (i + 1, j - 1)):
            if 0 <= x < self.M and 0 <= y < self.N:
                result += self.grid[x][y]
        return result

# Testing the solution
ns = neighborSum([[0,1,2], [3,4,5], [6,7,8]])
assert ns.adjacentSum(1) == 6
assert ns.adjacentSum(4) == 16
assert ns.diagonalSum(4) == 16
assert ns.diagonalSum(8) == 4

In [10]:
def shortestDistanceAfterQueries(N: int, queries: list[list[int]]) -> list[int]:
    def shortest_path() -> int:
        queue: collections.deque[tuple[int, int]] = collections.deque([(0, 0)])
        while queue:
            curr, dist = queue.popleft()
            for next_node in adjl[curr]:
                if next_node == N - 1:
                    return dist + 1
                else:
                    queue.append((next_node, dist + 1))

        return -1

    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for i in range(N):
        adjl[i].append(i + 1)

    result: list[int] = []
    for n1, n2 in queries:
        adjl[n1].append(n2)
        result.append(shortest_path())

    return result

# Testing the solution
assert shortestDistanceAfterQueries(5, [[2,4],[0,2],[0,4]]) == [3,2,1]
assert shortestDistanceAfterQueries(4, [[0,3],[0,2]]) == [1,1]

#### Matrix factorization GYM Practice: https://codeforces.com/gym/102644

In [11]:
def count_paths_brute(N: int, E: int, K: int, edges: list[tuple[int, int]]) -> int:
    # Convert edges into adjl
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n2].append(n1)

    # Do a 'cached' DFS
    MOD = 10 ** 9 + 7

    # Do a DFS traversal and cache the results
    dp: dict[tuple[int, int], int] = {(0, K + 1): 0}
    stack: list[tuple[int, int, int]] = [(i, 0, K) for i in range(1, N + 1)]
    while stack:
        curr, prev, k = stack[-1]
        if (curr, k) in dp or k == 0:
            if k == 0:
                dp[(curr, k)] = 1
            dp[(prev, k + 1)] = (dp[(prev, k + 1)] + dp[(curr, k)]) % MOD
            stack.pop()
        else:
            dp[(curr, k)] = 0
            for next_node in adjl[curr]:
                stack.append((next_node, curr, k - 1))

    return dp[(0, K + 1)]

# Testing the solution
assert count_paths_brute(3, 4, 2, [(1,2), (2,3), (3,1), (2,1)]) == 5
assert count_paths_brute(5, 10, 11, [(2,3), (4,2), (2,1), (2,4), (1,5), (5,2), (3,2), (3,1), (3,4), (1,2)]) == 21305

In [12]:
def count_paths(n_vertices: int, n_edges: int, K: int, edges: list[tuple[int, int]]) -> int:
    """
    Adj matrix raised to power K

    Each cell (i, j) denotes how many paths exist from node i + 1 to j + 1.
    Total paths would be the sum of all cells
    """

    def matrix_multiply(A: list[list[int]], B: list[list[int]]) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "invalid matrix dimensions"
        result: list[list[int]] = [[0 for j in range(M)] for i in range(Q)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % MOD

        return result

    def matrix_power(base: list[list[int]], exp: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while exp:
            if exp & 1:
                result = matrix_multiply(result, base)
            base = matrix_multiply(base, base)
            exp >>= 1

        return result

    # Return path count % MOD
    MOD = 10 ** 9 + 7
    adjm: list[list[int]] = [[0 for j in range(n_vertices)] for i in range(n_vertices)]
    for n1, n2 in edges:
        adjm[n1 - 1][n2 - 1] = 1

    # Raise Adjm to the power of K using matrix factorization
    result = matrix_power(adjm, K)
    return sum(sum(row) for row in result) % MOD

# Testing the solution
assert count_paths(3, 4, 2, [(1,2), (2,3), (3,1), (2,1)]) == 5
assert count_paths(5, 10, 11, [(2,3), (4,2), (2,1), (2,4), (1,5), (5,2), (3,2), (3,1), (3,4), (1,2)]) == 21305

In [13]:
def knight_paths(K: int) -> int:

    ij2n: typing.Callable[[int, int], int] = lambda x, y: 8 * x + y
    n2ij: typing.Callable[[int], tuple[int, int]] = lambda n: (n // 8, n % 8)

    def matrix_multiply(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "invalid dimensions"
        result: list[list[int]] = [[0 for j in range(Q)] for i in range(M)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod

        return result

    def matrix_power(base: list[list[int]], power: int, mod: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while power:
            if power & 1:
                result = matrix_multiply(result, base, mod)
            base = matrix_multiply(base, base, mod)
            power //= 2

        return result

    def mark_paths(n: int) -> list[int]:
        x1, y1 = n2ij(n)
        result: list[int] = []
        for x_delta in (-2, -1, 1, 2):
            for y_delta in (-2, -1, 1, 2):
                x2, y2 = x1 + x_delta, y1 + y_delta
                if 0 <= x2 < 8 and 0 <= y2 < 8 and abs(x_delta) != abs(y_delta):
                    result.append(ij2n(x2, y2))

        return result

    @functools.cache
    def generate_base() -> list[list[int]]:
        # Count of paths the knight can make starting at given point
        base: list[list[int]] = [[0 for j in range(65)] for i in range(65)]
        for i in range(64):
            for j in mark_paths(i):
                base[i][j] = 1

        # We store the cells from 0 - 63, 64 stores the answer
        for i in range(65):
            base[i][64] = 1

        return base

    base = generate_base()
    result = matrix_power(base, K + 1, 4294967296)
    return result[0][64]

# Testing the solution
assert knight_paths(1) == 3
assert knight_paths(2) == 15
assert knight_paths(6) == 17231

#### Codeforces Div 2 - 4th July 2024

In [14]:
def question_marks(N: int, answers: str) -> int:
    marks = 0
    correct: dict[str, int] = {ch: N for ch in "ABCD"}
    for answer in answers:
        if answer in correct:
            marks += 1
            correct[answer] -= 1
            if correct[answer] == 0:
                correct.pop(answer)

    return marks

# Testing the solution
assert question_marks(1, "ABCD") == 4
assert question_marks(2, "AAAAAAAA") == 2
assert question_marks(2, "AAAABBBB") == 4
assert question_marks(2, "????????") == 0
assert question_marks(3, "ABCABCABCABC") == 9
assert question_marks(5, "ACADC??ACAC?DCAABC?C") == 13

In [15]:
def parity_and_sum(N: int, nums: list[int]) -> int:
    # Split into odd / even
    odd: list[int] = []
    even: list[int] = []
    for n in nums:
        if n % 2 == 0:
            even.append(n)
        else:
            odd.append(n)

    # Sort odd and even
    odd.sort(reverse=False)
    even.sort(reverse=True)

    # Check the smallest in odd and largest in even
    moves = 0
    while odd and even:
        moves += 1
        if even[-1] < odd[-1]:
            odd.append(even.pop() + odd[-1])
        else:
            odd[-1] += even[0]

    return moves

# Testing the solution
assert parity_and_sum(4, [1,4,10,12]) == 4
assert parity_and_sum(5, [1,3,5,7,9]) == 0
assert parity_and_sum(4, [4,4,4,4]) == 0
assert parity_and_sum(3, [2,3,4]) == 2
assert parity_and_sum(4, [3,2,2,8]) == 4
assert parity_and_sum(6, [4,3,6,1,2,1]) == 3

In [16]:
def light_switches(N: int, K: int, install_time: list[int]) -> int:
    start, end = install_time[0], install_time[0] + K - 1
    for time in install_time[1:]:
        curr_start, curr_end = time, time + K - 1

        # Always ensure that start, end is the greater
        if start < curr_start:
            start, end, curr_start, curr_end = curr_start, curr_end, start, end

        # If they don't overlap, try to increase them to the nearest range
        if curr_end < start:
            factor = math.ceil((start - curr_end) / (2 * K))
            curr_start, curr_end = curr_start + (factor * 2 * K), curr_end + (factor * 2 * K)

        # Check if overlap
        start, end = max(start, curr_start), min(end, curr_end)

        if end < start:
            return -1

    return start

# Testing the solution
assert light_switches(3, 3, [6,2,1]) == 8
assert light_switches(7, 5, [14,34,6,25,46,7,17]) == 47
assert light_switches(4, 3, [2,3,4,5]) == -1
assert light_switches(4, 4, [2,3,4,5]) == 5
assert light_switches(4, 3, [3,4,8,9]) == 10
assert light_switches(1, 1, [1]) == 1
assert light_switches(6, 5, [40,80,99,60,90,50]) == 100
assert light_switches(6, 5, [64,40,50,68,70,10]) == -1

Upsolving CF Div - 2

In [17]:
def medimize(N: int, K: int, nums: list[int]) -> int:
    """
    Use binary search to find the maximum median - straight forward
    Use DP to check if a given median can be formed or not - Convert nums into binary nums of 1 or -1

    After removing K elements, we would be left with `K if N % K == 0 else N % K elements'
    The recursive solution would be a pick / nopick, if no pick we skip k elements

    Final array would contain elements:

    ai0, ai1, ai2, ... aim
    i0 = 0 mod k
    i1 = 1 mod k
    ..
    im = (m - 1) mod k

    Since we have 'B' with either -1 or 1, if the sum(b) > 0 then true else false

    In check function, why > 0?
    We are not checking if the median itself is possible, we are simply checking if a value greater
    than or equal to median is possible to be the answer

    For instance: [5,1,6,9,4,8,3,2] | K = 4; here 3 itself is not possible to be the median but check(3) is still true
    We are checking if a value greater than or equal to 3 can be the answer.

    """
    def check_brute(median: int, B: list[int]) -> bool:
        @functools.cache
        def backtrack(curr: int, R: int) -> float:
            if R == 0:
                return 0
            elif curr >= N:
                return -math.inf
            else:
                return max(B[curr] + backtrack(curr + 1, R - 1), backtrack(curr + K, R))

        remaining = K if N % K == 0 else N % K
        return backtrack(0, remaining) > 0

    def check(median: int, B: list[int]) -> bool:
        dp: list[int] = [0 for i in range(len(B))]
        dp[0] = B[0]
        for i in range(1, len(B)):
            if i % K == 0:
                dp[i] = max(dp[i - K], B[i])
            else:
                dp[i] = dp[i - 1] + B[i]
                if i > K:
                    dp[i] = max(dp[i], dp[i - K])

        return dp[-1] > 0

    low, high = min(nums), max(nums)
    while low <= high:
        mid = (low + high) // 2
        if check(mid, [-1 if n < mid else 1 for n in nums]):
            low = mid + 1
        else:
            high = mid - 1

    return high

# Testing the solution
assert medimize(8, 2, [7,1,2,6,8,3,4,5]) == 6
assert medimize(4, 5, [3,4,5,6]) == 4
assert medimize(7, 1, [5,9,2,6,5,4,6]) == 9
assert medimize(5, 3, [3,2,5,6,4]) == 4
assert medimize(4, 3, [3,9,9,2]) == 3

In [18]:
def scripted_robot(N: int, K: int, W: int, H: int, script: str) -> int:
    """
    Iterate once through the script and store the valid positions that we end up at after executing each script.

    To make things easier, we can observe that if we left the robot to move undetered, it would basically be the same as
    the robot following the rules after |2 * H| or |2 * W| steps along the y axis and x axis respectively.

    Next assume after executing the script we end up at some position that is dx, dy from origin.
    Inorder to hit (0, 0) we need -dx, -dy so we count number of times we had hit (-dx, -dy) when we first executed the script.

    For next iteration we would need to check for -2dx, -2dx.. -3dx, -3dy and so on until K

    Note that in other languages x % mod would return a negative number if x is negative, so in tutorials we would see (x + mod) % mod
    In python x % mod would be negative only when mod itself is negative
    """

    delta_mapping: dict[str, tuple[int, int]] = {"L": (0, -1), "R": (0, 1), "U": (1, 0), "D": (-1, 0)}
    counts: collections.defaultdict[tuple[int, int], int] = collections.defaultdict(int)
    mod_x, mod_y = 2 * W, 2 * H

    # Compute positions where the robot would end up inside the box
    x, y = 0, 0
    for command in script:
        # Compute the position post command, undetered by boundaries
        delta = delta_mapping[command]
        x, y = x + delta[1], y + delta[0]

        # Making use of the `mirrored` property
        x, y = x % mod_x, y % mod_y
        counts[(x, y)] += 1

    # For each k in 0..k-1, compute the number of times -i * dx, -i * dy could be found
    # Why check for -dx, -dy? Because dx + (-dx), dy + (-dy) would give us (0, 0)
    # which is what we are tracking
    dx, dy = x, y
    result = 0
    for i in range(K):
        x = (-i * dx) % mod_x
        y = (-i * dy) % mod_y
        result += counts[(x, y)]

    return result

# Testing the solution
assert scripted_robot(2, 2, 2, 2, "UR") == 0
assert scripted_robot(4, 2, 1, 1, "LLDD") == 4
assert scripted_robot(6, 3, 3, 1, "RLRRRL") == 3
assert scripted_robot(5, 5, 3, 3, "RUURD") == 0
assert scripted_robot(7, 5, 3, 4, "RRDLUUU") == 1

LC Weekly upsolving - Shortest dist after road additions - II

In [19]:
def shortestDistanceAfterQueriesII(N: int, queries: list[list[int]]) -> list[int]:
    """
    Imagine the roads as a linked list, the idea for each query is to
    remove all edges between (i, j) (non inclusive)
    """
    connections: dict[int, int] = {i: i + 1 for i in range(N - 1)}
    result: list[int] = []
    for n1, n2 in queries:

        # Ensure n1 is not deleted by a previous query
        # And ensure that the current query is not lesser
        # than a previously encountered query
        # For eg: we encountered (2, 5) already, (2, 3) is not useful
        if n1 in connections and connections[n1] < n2:
            curr = connections[n1]
            while curr < n2:
                curr = connections.pop(curr)
            connections[n1] = n2

        result.append(len(connections))

    return result

# Testing the solution
assert shortestDistanceAfterQueriesII(5, [[2,4],[0,2],[0,4]]) == [3,2,1]
assert shortestDistanceAfterQueriesII(4, [[0,3],[0,2]]) == [1,1]

LC Daily Problem

In [20]:
def kthDistinct(arr: list[str], k: int) -> str:
    """
    Python has its useful quirks too.
    In dictionary the insertion order is preserved.

    For arr containing a lot of duplicates, we can quickly skip them since
    we would only encounter them once in the freq
    """
    freq: collections.Counter[str] = collections.Counter(arr)
    for string in freq:
        if freq[string] == 1:
            k -= 1
        if k == 0:
            return string

    return ""

# Testing the solution
assert kthDistinct(["d","b","c","b","c","a"], k = 2) == "a"
assert kthDistinct(["aaa","aa","a"], k = 1) == "aaa"
assert kthDistinct(["a","b","a"], k = 3) == ""

Striver's CP Sheet

In [21]:
def wet_shark_and_blocks_brute(N: int, B: int, K: int, X: int, nums: list[int]) -> int:
    # Return ans mod
    MOD = 10 ** 9 + 7

    # Convert block of nums into freq
    freq: list[int] = [0 for i in range(10)]
    for n in nums:
        freq[n] += 1

    # return backtrack(0, 0)
    dp: list[int] = [0 if i != K else 1 for i in range(X)]
    for blocks in range(B):
        next_dp: list[int] = [0 for i in range(X)]
        for total in range(X):
            for next_ in range(1, 10):
                next_dp[total] = (next_dp[total] + freq[next_] * dp[(total * 10 + next_) % X]) % MOD
        dp = next_dp

    return dp[K]

# Testing the solution
assert wet_shark_and_blocks_brute(12, 1, 5, 10, [3,5,6,7,8,9,5,1,1,1,1,5]) == 3
assert wet_shark_and_blocks_brute(3, 2, 1, 2, [6,2,2]) == 0
assert wet_shark_and_blocks_brute(3, 2, 1, 2, [3,1,2]) == 6

In [22]:
def wet_shark_and_blocks(N: int, B: int, K: int, X: int, nums: list[int]) -> int:
    def matrix_multiply(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "Invalid dimensions provided."
        result: list[list[int]] = [[0 for j in range(Q)] for i in range(M)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod
        return result

    def matrix_power(base: list[list[int]], exp: int, mod: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while exp:
            if exp & 1:
                result = matrix_multiply(result, base, mod)
            base = matrix_multiply(base, base, mod)
            exp >>= 1
        return result

    # Answer can be very large
    MOD = 10 ** 9 + 7

    # Create a freq array
    freq: list[int] = [0 for i in range(10)]
    for n in nums:
        freq[n] += 1

    # Create transformation matrix
    base: list[list[int]] = [[0 for j in range(X)] for i in range(X)]
    for total in range(X):
        for next_ in range(1, 10):
            new_total = (total * 10 + next_) % X
            base[new_total][total] = (base[new_total][total] + freq[next_]) % MOD

    result = matrix_power(base, B, MOD)
    return result[K][0]


# Testing the solution
assert wet_shark_and_blocks(12, 1, 5, 10, [3,5,6,7,8,9,5,1,1,1,1,5]) == 3
assert wet_shark_and_blocks(3, 2, 1, 2, [6,2,2]) == 0
assert wet_shark_and_blocks(3, 2, 1, 2, [3,1,2]) == 6

#### CF Div 4 - 6th Aug 2024

In [23]:
def digit_sum(n: int) -> int:
    return n // 10 + n % 10

# Testing the solution
assert digit_sum(77) == 14
assert digit_sum(10) == 1
assert digit_sum(48) == 12

In [24]:
def card_game(a1: int, a2: int, b1: int, b2: int) -> int:
    count = 0
    if (a1 > b1 and a2 >= b2) or (a2 > b2 and a1 >= b1):
        count += 2
    if (a1 > b2 and a2 >= b1) or (a2 > b1 and a1 >= b2):
        count += 2
    return count

# Testing the solution
assert card_game(3, 8, 2, 6) == 2
assert card_game(1, 1, 1, 1) == 0
assert card_game(10, 10, 2, 2) == 4
assert card_game(1, 1, 10, 10) == 0
assert card_game(3, 8, 7, 2) == 2

In [25]:
def showering(N: int, S: int, M: int, tasks: list[tuple[int, int]]) -> bool:
    tasks.sort()
    prev_end = 0
    for start, end in tasks:
        if start - prev_end >= S:
            return True
        prev_end = end

    return M - prev_end >= S

# Testing the solution
assert showering(3, 3, 10, [(3,5), (6,8), (9,10)]) == True
assert showering(3, 3, 10, [(1,2), (3,5), (6,7)]) == True
assert showering(3, 3, 10, [(1,2), (3,5), (6,8)]) == False
assert showering(3, 4, 10, [(1,2), (6,7), (8,9)]) == True

In [26]:
def slavic_exam(S: str, T: str) -> str:
    i, j, s_length, t_length = 0, 0, len(S), len(T)
    result: list[str] = []
    for i in range(s_length):
        result.append(S[i] if S[i] != "?" else T[j] if j < t_length else "a")
        if j < t_length and (S[i] == T[j] or S[i] == "?"):
            j += 1

    return "".join(result) if j == t_length else ""

# Testing the solution
assert slavic_exam("?????", "xbx") == "xbxaa"
assert slavic_exam("ab??e", "abcde") == "abcde"
assert slavic_exam("ayy?x", "a") == "ayyax"
assert slavic_exam("ab??e", "dac") == ""
assert slavic_exam("paiu", "mom") == ""

#### Upsolving CF Div 4 from yesterday

In [27]:
def triple_operations(queries: list[tuple[int, int]]) -> list[int]:
    """
    Attempt to make use of math.log(a, b) failed, reason being the precision point
    error in programming languages. math.log(243, 3) => 4.9999999 when it should be 5

    math.log(a, b) btw uses math.ln for its calculations, not reliable, hence approach
    below where we cache the cost
    """
    # Precompute the cost
    MAX = 2 * 10 ** 5
    moves: list[int] = [0]
    cumulative_moves: list[int] = [0]
    for i in range(1, MAX + 1):
        moves.append(1 + moves[i // 3])
        cumulative_moves.append(moves[-1] + cumulative_moves[-1])

    result: list[int] = []
    for L, R in queries:
        result.append(2 * moves[L] + cumulative_moves[R] - cumulative_moves[L])

    return result

# Testing the solution
assert triple_operations([(1, 3), (2, 4), (19, 84), (243, 245), (242, 243)]) == [5, 6, 263, 24, 16]

In [28]:
def expected_median(N: int, K: int, nums: list[int]) -> int:
    # Answer modulo mod
    MOD = 10 ** 9 + 7

    # Precompute factorials and Inv factorials
    # Inv Factorial using fermat's little theorem: 1 / a! => a! ** (mod - 2)
    factorials: list[int] = [1]
    inv_factorials: list[int] = [1]
    MAX = 2 * 10 ** 5
    for i in range(1, MAX + 1):
        factorials.append((i * factorials[-1]) % MOD)
        inv_factorials.append(pow(factorials[-1], MOD - 2, MOD))

    # Application of Fermat's little theorem - `pow` part
    def comb(n: int, r: int) -> int:
        # Python can calculate negative powers already
        # result = (factorials[n] * pow(factorials[r] * factorials[n - r], -1, MOD)) % MOD
        result = (factorials[n] * inv_factorials[r] * inv_factorials[n - r]) % MOD
        return result

    # Count 1s present
    total_ones = sum(nums)

    # Count number of subsequences having 1s > K // 2
    total_ways = 0
    for x in range(K // 2 + 1, min(total_ones, K) + 1):
        if K - x <= N - total_ones:
            total_ways = (total_ways + comb(total_ones, x) * comb(N - total_ones, K - x)) % MOD

    return total_ways

# Testing the solution
assert expected_median(4, 3, [1,0,0,1]) == 2
assert expected_median(5, 1, [1,1,1,1,1]) == 5
assert expected_median(5, 5, [0,1,0,1,0]) == 0
assert expected_median(6, 3, [1,0,1,0,1,1]) == 16
assert expected_median(4, 3, [1,0,1,1]) == 4
assert expected_median(5, 3, [1,0,1,1,0]) == 7
assert expected_median(2, 1, [0,0]) == 0
assert expected_median(34, 17, [1] * 34) == 333606206

#### Ruler CF

In [29]:
def ruler() -> None:
    "Using Ternery search, split search space into 3 equal chunks"
    T = int(input())
    for i in range(T):
        low, high = 2, 999
        while low <= high:
            mid1, mid2 = (2 * low + high) // 3, (2 * high + low) // 3
            print(f"? {mid1} {mid2}")
            area = int(input())
            if area == mid1 * mid2:
                low = mid2 + 1
            elif area == mid1 * (mid2 + 1):
                low, high = mid1 + 1, mid2 - 1
            else:
                high = mid1 - 1
        print(f"! {low}")

LC POTD: https://leetcode.com/problems/integer-to-english-words/submissions/1348041229

In [30]:
def number_to_words(N: int) -> str:
    def to_words(N: int) -> str:
        results: list[str] = []
        if N // 100 > 0:
            results.append(f"{mapping[N // 100]} {mapping[100]}")
            N %= 100
        if N > 0:
            if N < 21:
                results.append(mapping[N])
            else:
                results.append(f"{mapping[N // 10 * 10]} {mapping[N % 10]}" if N % 10 else mapping[N // 10 * 10])

        return " ".join(results)

    # Building blocks of english wording of numbers
    mapping: dict[int, str] = {
        1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five",
        6: "Six", 7: "Seven", 8: "Eight", 9: "Nine",

        11: "Eleven", 12: "Twelve", 13: "Thirteen", 14: "Fourteen",
        15: "Fifteen", 16: "Sixteen", 17: "Seventeen", 18: "Eighteen",
        19: "Nineteen",

        20: "Twenty", 30: "Thirty", 40: "Forty", 50: "Fifty", 60: "Sixty",
        70: "Seventy", 80: "Eighty", 90: "Ninety",

        10: "Ten", 100: "Hundred", 1000: "Thousand", 1000000: "Million",
        1000000000: "Billion", 0: ""
    }

    # Idea is to split into groupings of 3
    groups: list[int] = []
    while N:
        groups.append(N % 1000)
        N //= 1000
    groups.reverse()

    if groups:
        words: list[str] = list(map(to_words, groups))
        curr = 1
        for i in range(-2, -(len(words) + 1), -1):
            curr *= 1000
            if words[i]:
                words[i] = f"{words[i]} {mapping[curr]}"

        # Remove any ""
        words = list(filter(lambda x: len(x), words))

    else:
        words = ["Zero"]

    result = " ".join(words)
    return result

# Testing the solution
assert number_to_words(30) == "Thirty"
assert number_to_words(100) == "One Hundred"
assert number_to_words(0) == "Zero"
assert number_to_words(2147483647) == "Two Billion One Hundred Forty Seven Million Four Hundred Eighty Three Thousand Six Hundred Forty Seven"
assert number_to_words(123) == "One Hundred Twenty Three"
assert number_to_words(12345) == "Twelve Thousand Three Hundred Forty Five"

#### CF Practice - Trie

In [31]:
def vasiley_multiset() -> None:
    BIT_LENGTH = 32

    class Node:
        one: 'Node|None'
        zero: 'Node|None'
        count: int

        def __init__(self, count: int) -> None:
            self.count = count
            self.one = None
            self.zero = None

        def __str__(self) -> str:
            return f"count: {self.count} ({self.zero} | {self.one})"

    def insert(curr: Node, bits: list[int], pos: int) -> None:
        curr.count += 1
        if pos < BIT_LENGTH:
            next_ = curr.one if bits[pos] else curr.zero
            if not next_:
                next_ = Node(0)
            if bits[pos]:
                curr.one = next_
            else:
                curr.zero = next_
            insert(next_, bits, pos + 1)

    def delete(curr: Node, bits: list[int], pos: int) -> None:
        curr.count -= 1
        if pos < BIT_LENGTH:
            next_: Node = curr.one if bits[pos] else curr.zero # type: ignore
            delete(next_, bits, pos + 1)
            if not next_.count:
                if bits[pos]:
                    curr.one = None
                else:
                    curr.zero = None

    def search(root: Node, bits: list[int]) -> int:
        result: int = 0
        pos, curr = 0, root
        while pos < BIT_LENGTH:
            if not curr.zero or (not bits[pos] and curr.one):
                result += 2 ** (BIT_LENGTH - pos - 1)
                curr = curr.one # type: ignore
            else:
                curr = curr.zero
            pos += 1

        return result

    # Read input and process queries
    Q = int(input())
    root: Node = Node(0)
    insert(root, list(map(int, "0".zfill(BIT_LENGTH))), 0)
    for i in range(Q):
        inp = input().split()
        op, num = inp[0], int(inp[1])
        bits = list(map(int, bin(num)[2:].zfill(BIT_LENGTH)))
        if op == "+":
            insert(root, bits, 0)
        elif op == "-":
            delete(root, bits, 0)
        else:
            print(num ^ search(root, bits))

In [32]:
def perfect_security(N: int, encs: list[int], pks: list[int]) -> list[int]:
    """
    Given the pk and encrypted message, return lexiographically smallest message.

    Idea is to use a Trie data structure.
    Initialize all keys into the root. As we go go through enc, make sure we find pki such that it is 'closest' to enci.
    Closest as in, starting from the most significant bit - 1s, 0s match as close as possible. This way xor result would be smallest.
    Once a key is picked, remove that key from the trie.
    """

    BIT_LENGTH = 32

    class Node:
        def __init__(self, count: int = 0) -> None:
            self.count: int = count
            self.one: 'Node|None' = None
            self.zero: 'Node|None' = None

    def insert(root: Node, pk: int) -> None:
        curr = root
        for pos in range(BIT_LENGTH - 1, -1, -1):
            curr.count += 1
            bit = (pk >> pos) & 1
            next_ = (curr.one if bit else curr.zero) or Node()
            if bit:
                curr.one = next_
            else:
                curr.zero = next_
            curr = next_
        curr.count += 1

    def search(curr: Node, enc: int) -> int:
        """Search for the closest matching pk and delete it"""
        result = 0
        for pos in range(BIT_LENGTH - 1, -1, -1):
            curr.count -= 1
            bit = (enc >> pos) & 1
            if (not curr.zero or curr.zero.count == 0) or (bit and curr.one and curr.one.count > 0):
                result |= 1 << pos
                curr = curr.one # type: ignore
            else:
                curr = curr.zero
        curr.count -= 1

        return result

    # Insert all permuted keys into trie
    root = Node()
    for pk in pks:
        insert(root, pk)

    # Get enc message
    result: list[int] = []
    for enc in encs:
        result.append(enc ^ search(root, enc))

    return result

# Testing the solution
assert perfect_security(5, [12,7,87,22,11], [18,39,9,12,16]) == [0,14,69,6,44]
assert perfect_security(5, [8,4,13], [17,2,7]) == [10,3,28]
assert perfect_security(10, [331415699,278745619,998190004,423175621,42983144,166555524,843586353,802130100,337889448,685310951], [226011312,266003835,342809544,504667531,529814910,684873393,817026985,844010788,993949858,1031395667]) == [128965467,243912600,4281110,112029883,223689619,76924724,429589,119397893,613490433,362863284]

In [ ]:
def beautiful_subarrays_brute(N: int, K: int, nums: list[int]) -> int:
    """
    Do a prefix sum for each invalid k: 1...K-1.
    Invalid counts are taken since the maximum valid K can be very large.

    Time: O(N x K)
    """

    def countK(k_: int) -> int:
        prefix: collections.defaultdict[int, int] = collections.defaultdict(int, {0: 1})
        xor = counts = 0
        for i in range(N):
            xor ^= nums[i]
            counts += prefix[xor ^ k_]
            prefix[xor] += 1
        return counts

    total_counts = N * (N + 1) // 2
    invalid_counts = sum(countK(k) for k in range(K))
    return total_counts - invalid_counts

# Testing the solution
assert beautiful_subarrays_brute(3, 1, [1,2,3]) == 5
assert beautiful_subarrays_brute(3, 2, [1,2,3]) == 3
assert beautiful_subarrays_brute(3, 3, [1,2,3]) == 2